# Teoria das filas

**Do Wikipedia**: *A teoria das filas é um ramo da probabilidade que estuda a formação de filas, através de análises matemáticas precisas e propriedades mensuráveis. Ela provê modelos para demonstrar previamente o comportamento de um sistema que ofereça serviços cuja demanda cresce aleatoriamente, tornando possível dimensioná-lo de forma a satisfazer os clientes e ser viável economicamente para o provedor do serviço, evitando desperdícios e gargalos.*

![alt text](https://keep.google.com/u/0/media/v2/1bkGCdfS7yWpf3XiXJT1Iyj8rd-VtYTVjgIp5kT1kjVVS3QFhACl7g1rd0rLznXlsw8AYyQ/1I2cr9mu_RtKdHtKxsuGzttE703SEZMt5I5YJE3RaifWbzPBz-BxcrH-oG8KpL47KRECCqw?accept=image/gif,image/jpeg,image/jpg,image/png,image/webp,audio/aac&sz=2031)

In [0]:
!pip install simpy

In [0]:
import simpy
import random

## Distribuição das chegadas e dos tempos

De uma forma bastante resumida imagine uma variável aleatória Poisson, onde temos a contagem do número de ocorrências em um intervalo. Suponha agora que estejamos interessados em verificar a probabilidade do tempo transcorrido entre duas ocorrências consecutivas. Essa última é considerada uma variável aleatória exponencial.

Essa distribuição contínua que pode ser utilizada para descrever as probabilidades envolvidas no tempo que decorre para que um determinado evento aconteça. Existe uma conexão muito próxima entre a distribuição exponencial e a de Poisson. Ou seja, é Utilizada para descrever o tempo entre as ocorrências de sucessivos eventos de uma distribuição de Poisson.As relações entre as distribuições podem ser associadas a um processo estocástico, chamado de processo de poisson.

![alt text](https://www.inf.ufsc.br/~andre.zibetti/probabilidade/figures/exponencial-Rstudio-3-1.png)

In [0]:
DISPTALHAO = 5.0         # Um talhão é liberado para corte a cada 5 dias. Segue distribuição exponencial.
TEMPOCOLHEITA = 20.0     # Em média, um talhão leva 20 dias para ser colhido.
PRODUCAO = []
FILA = []

## Entidades do modelo

1. Gerador de talhão
2. Recurso

In [0]:
def listaTalhao(env):
  listaNovo = 0
  while True:
    yield env.timeout(random.expovariate(1.0/DISPTALHAO))
    listaNovo += 1
    print('%.1f Liberado talhao %d para colheita' % (env.now, listaNovo))
    FILA.append('Talhão ' + str(listaNovo))

    env.process(colheTalhao(env, 'Talhão %d' % listaNovo, moduloColheita))

In [0]:
def colheTalhao(env, nome, moduloColheita):
  request = moduloColheita.request()

  yield request
  print('%.1f Inicia a colheita de %s' % (env.now, nome))

  yield env.timeout(random.expovariate(1.0/TEMPOCOLHEITA))
  producao = random.normalvariate(15000.0, 5000.0)
  PRODUCAO.append(producao)
  print('%.1f Termina a colheita de %s e produziu %d m3' % (env.now, nome, producao))
  FILA.remove(nome)

  yield moduloColheita.release(request)

## Ambiente de simulação

In [0]:
env = simpy.Environment()
moduloColheita = simpy.Resource(env, capacity=2)
env.process(listaTalhao(env))
env.run(until=365)

0.8 Liberado talhao 1 para colheita
0.8 Inicia a colheita de Talhão 1
4.5 Liberado talhao 2 para colheita
4.5 Inicia a colheita de Talhão 2
8.8 Liberado talhao 3 para colheita
10.9 Liberado talhao 4 para colheita
12.0 Liberado talhao 5 para colheita
20.3 Liberado talhao 6 para colheita
20.8 Liberado talhao 7 para colheita
21.7 Liberado talhao 8 para colheita
34.1 Liberado talhao 9 para colheita
34.4 Termina a colheita de Talhão 1 e produziu 13635 m3
34.4 Inicia a colheita de Talhão 3
40.0 Liberado talhao 10 para colheita
42.2 Liberado talhao 11 para colheita
44.3 Liberado talhao 12 para colheita
49.2 Termina a colheita de Talhão 3 e produziu 17752 m3
49.2 Inicia a colheita de Talhão 4
50.0 Liberado talhao 13 para colheita
56.7 Termina a colheita de Talhão 2 e produziu 10598 m3
56.7 Inicia a colheita de Talhão 5
64.4 Liberado talhao 14 para colheita
64.6 Liberado talhao 15 para colheita
68.5 Termina a colheita de Talhão 5 e produziu 14917 m3
68.5 Inicia a colheita de Talhão 6
68.6 Termi

In [0]:
print('A produção anual foi de', round(sum(PRODUCAO), 1), 'm³')
print(FILA)

A produção anual foi de 617593.3 m³
['Talhão 40', 'Talhão 41', 'Talhão 42', 'Talhão 43', 'Talhão 44', 'Talhão 45', 'Talhão 46', 'Talhão 47', 'Talhão 48', 'Talhão 49', 'Talhão 50', 'Talhão 51', 'Talhão 52', 'Talhão 53', 'Talhão 54', 'Talhão 55', 'Talhão 56', 'Talhão 57', 'Talhão 58', 'Talhão 59', 'Talhão 60', 'Talhão 61', 'Talhão 62', 'Talhão 63', 'Talhão 64', 'Talhão 65', 'Talhão 66']
